# Deeply Embedded DSLs in Python

Taking language from ["Deep and Shallow Embeddings"](https://www.cs.ox.ac.uk/people/jeremy.gibbons/publications/embedding-short.pdf) by Jeremy Gibbons, domain specific languages thare are embedded in a host language, come in two flavors: shallow and deep embedding.


## Shallow

In [22]:
import dataclasses
import typing

@dataclasses.dataclass
class NaturalShallow:
    value: int

    def __add__(self, other: "NaturalShallow") -> "NaturalShallow":
        return NaturalShallow(self.value + other.value)


In [53]:
NaturalShallow(3) + NaturalShallow(4) + NaturalShallow(2)

NaturalShallow(value=9)

## Deep

In [50]:
import dataclasses
import typing



@dataclasses.dataclass
class NaturalAdd:
    left: "NaturalDeep"
    right: "NaturalDeep"

    def execute(self) -> int:
        return self.left.execute() + self.right.execute()

    
@dataclasses.dataclass
class NaturalDeep:
    value: typing.Union[int, NaturalAdd]

    def __add__(self, other: "NaturalDeep") -> "NaturalAdd":
        return NaturalDeep(NaturalAdd(self, other))
    
    def execute(self) -> int:
        if isinstance(self.value, int):
            return self.value
        return self.value.execute()


In [51]:
NaturalDeep(3) + NaturalDeep(4) + NaturalDeep(1)

NaturalDeep(value=NaturalAdd(left=NaturalDeep(value=NaturalAdd(left=NaturalDeep(value=3), right=NaturalDeep(value=4))), right=NaturalDeep(value=1)))

In [52]:
(NaturalDeep(3) + NaturalDeep(4) + NaturalDeep(1)).execute()

8

We see that the shallow embedding does the execution when the function is called. This is like most APis in built into Python, including NumPy's. It is nice and simple to understand.

But what do we get by having a deep embedding? Well one thing we get is the ability to define different interpretions of some execution. Here we have `execute` defined, but we also get (for free) a string representation of the execution. So already this can be useful when understanding what the API will do.

But the real power we get here is that we turn the problem from "What do I do when I call this function?" to "I have this data structure representing some call. Now what can I do with it?" So we can see how we get the same sort of power Lisp users talk about, and we can easily add macro-like behavior without any changes to the language.

For example, we could now go into the execution data structures and move things around as we wish, with all the power of Python itself.

A library like multipledispatch could be useful in creating a shallowly embedded DSL, but it won't get you all the way to a deeply embedded DSL, because it still ties the executiong of a function to it's call. **The whole point of a deeply embedded DSL is to break up those two steps, where the user says "I want to call this function with these arguments" and the actual execution of that function with those arguments.** By putting some abstraction between those steps, we can do a lot of interesting things, like optimize the expression, translate it to some other computation backend, or try to prove properties about it.

## What we need

Libraries like `ibis` or `dask.array` are using this strategy effectively today to take one API (Pandas, NumPy) and map it into a different execution context (SQL, computation graph). 

On the other end, efforts like NumPy's API exposure is about enabling more libraries like this to exist, by allowing libraries to override how NumPy methods are called.

What is missing? Well it's hard for these packages to work with each other. They exist in seperate worlds and can't share code. Numba, for example, has some useful code on optimizing certain NumPy calls. There is no reason those couldn't be reused when you create NumPy expression in `dask.array`. Why aren't they shared? Well they each use different representations of what a NumPy expression is. 

So, I am curious what a shared based could like to support these different use cases and encourage best practices, innovation, and sharing of work.

In the NumPy case, this would mean first being able to turn a number of NumPy calls into a deeply embedded representation, that would be agnostic to how it would be executed. Then there would be libraries that would convert between this representation and Numba IR or Dask expressions. So what is at the core?

We need a way to express a call to function, without actually calling it. So let's create that:

In [60]:
@dataclasses.dataclass
class Call:
    function: typing.Callable
    args: typing.Tuple["NaturalDeep2", ...]

Now let's rewrite our natural number implementation using it:

In [61]:
@dataclasses.dataclass
class NaturalDeep2:
    value: typing.Union[int, Call]

    def __add__(self, other: "NaturalDeep2") -> "Call":
        return NaturalDeep2(Call(NaturalDeep2.__add__, (self, other)))
    
    def execute(self) -> int:
        if isinstance(self.value, int):
            return self.value
        return self.value.args[0].execute() + self.value.args[1].execute()


In [62]:
NaturalDeep2(123) + NaturalDeep2(456) + NaturalDeep2(123) 

NaturalDeep2(value=Call(function=<function NaturalDeep2.__add__ at 0x10f01e840>, args=(NaturalDeep2(value=Call(function=<function NaturalDeep2.__add__ at 0x10f01e840>, args=(NaturalDeep2(value=123), NaturalDeep2(value=456)))), NaturalDeep2(value=123))))

In [63]:
(NaturalDeep2(123) + NaturalDeep2(456) + NaturalDeep2(123)).execute()

702

This looks good. However, let's seperate the execution from the class, so we can add multuple versions without modifying the class:

In [68]:
@dataclasses.dataclass
class NaturalDeep3:
    value: typing.Union[int, Call]

    def __add__(self, other: "NaturalDeep2") -> "Call":
        return NaturalDeep2(Call(NaturalDeep2.__add__, (self, other)))


def execute(natural: NaturalDeep3) -> int:
    if isinstance(natural.value, int):
        return natural.value
    return execute(natural.value.args[0]) + execute(natural.value.args[1])


In [69]:
execute(NaturalDeep3(1) + NaturalDeep3(2))

3

Now we can add another function, like `cost` that computes the expected cost of the computation before it executes:

In [72]:
def cost(natural: NaturalDeep3) -> int:
    if isinstance(natural.value, int):
        return 0
    return 1 + cost(natural.value.args[0]) + cost(natural.value.args[1])


In [75]:
cost(NaturalDeep3(1) + NaturalDeep3(2) + NaturalDeep3(1))

2

You might notice that both `cost` and `execute` share a similar structure, where they have a base case for the leaf node, and an accumalator over the children nodes. We can factor out this pattern with a general `fold` high order function.

In [76]:
T = typing.TypeVar("T")

def fold_natural(leaf_fn: typing.Callable[[int], T], call_fn: typing.Callable[[typing.Callable, typing.Iterable[T]], T], natural: NaturalDeep3) -> T:
    if isinstance(natural.value, int):
        return leaf_fn(natural.value)

    partial_fold = lambda other_natural: fold_natural(leaf_fn, call_fn, other_natural)
    return call_fn(natural.value.function, map(partial_fold, natural.value.args))

Now we can define execute and cost using this higher order function:

In [84]:
import functools

cost2 = functools.partial(fold_natural, lambda _: 0, lambda fn, costs: 1 + sum(costs))
evaluate2 = functools.partial(fold_natural, lambda v: v, lambda fn, values: sum(values))

In [86]:
cost2(NaturalDeep3(1) + NaturalDeep3(2) + NaturalDeep3(1))


2

In [87]:
evaluate2(NaturalDeep3(10) + NaturalDeep3(2))

12

Great! Now we are really making some headway in abstracting out the exection.

To do this for real, we would have to make `evaluate` have some form of registration and multiple dispatch. We could do this many different ways, with pattern matching, replacement, or a more tradiotional approach of defining different handler functions for each operation.

---

Now there is another way we could defime DSLs. Which is how Numba and Autograph work. They don't build up expression, they analyze the CFG. Why? Because not all operations in Python are expressions.

For example with the approach here, you can override function application, but you cannot override control flow. For example, there would be no way to implement something like this:

```python
x: NaturalDeep3

if x < NaturalDeep3(10):
    y = x - 100
else:
    y = x + 30
```

We could do it by introducing a special boolean type and an if expression, like this:

```python
y = (x < NaturalDeep3(10)).if_(x - 100, x + 30)
```

But generally translating between the first form and the second is pretty complicated!

What if there is a return statement in the if clause? How would you compute a proper expression for this?

This is the problem the ["The 800 Pound Python in the Machine Learning Room"](https://paperpile.com/app/p/f543bd75-2192-0a48-ae1d-c3d28c410439) paper tackles. It provides a function wrapper, that transforms the AST to remove
control flow like that and turn it into function calls, so that you can override it.

Generally, this problem comes down to translating between a control flow graph and an expression tree. This is where the ["Optimizing compilation with the Value State Dependence Graph"](https://paperpile.com/app/p/da3d91a8-5c51-0d76-ba27-847c8a7eb41a) paper comes into play. Our representation here in Python is a subset of  the "Value State Dependence Graph" representation.

So what does this mean for a generic DSL library in Python? Well it's much easier to think about execution on an expression tree/graph, like the one we introduced. Pattern matching can be used to implement partial evaluation and use that for computation.

But, it's nice to write imperative code and its the common idiom in python to write control flow heavy code.   But you might notice that actually in the NumPy world, it is common to use it's abstractions for control flow, not to use Python's native control flow.

So as a start, we can just support that kind of API and forgot about going from a CFG. 

However eventually, we need to start with, and end up with, a CFG. Because if we wanna compile one of these expressions back to Python, or to another language, which isn't based on expression trees, then we will need one.

So to be generally useful, we should figure out the right abstractions for translating these types of tree expressions to and from CFGs. This is not new work, it has been done in the compiler community. What it takes is understanding that work enough to understand what abstractions to bring over here to Python. 